In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from pathlib import Path

import plotly.io as pio
pio.renderers.default = "notebook"

import sys
sys.path.append("../")
from building_era5 import load_IBTrACS

import matplotlib.pyplot as plt

# IBTrACS

In [ ]:
PROCESSED_DIR = Path("../../data/processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

path_ibtracs = PROCESSED_DIR / "other/ibtracs_usa_20251216.csv"

df_ibtracs = load_IBTrACS(path_ibtracs, years=[2022, 2024])

df_ibtracs["time_stamp"] = pd.to_datetime(df_ibtracs["time_stamp"], errors="coerce")
df_ibtracs = df_ibtracs.dropna(subset=["time_stamp"])

years = sorted(df_ibtracs["time_stamp"].dt.year.unique())

for year in years:
    df_y = df_ibtracs[df_ibtracs["time_stamp"].dt.year == year]

    counts = (
        df_y.groupby("basin")
        .size()
        .sort_values(ascending=False)
    )

    plt.figure(figsize=(8, 4))
    counts.plot(kind="bar")
    plt.title(f"Number of IBTrACS observations per basin – {year}")
    plt.ylabel("Number of observations")
    plt.xlabel("Basin")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()


In [ ]:
hour_counts = (
    df_ibtracs["time_stamp"]
    .dt.hour
    .value_counts()
    .sort_index()
)

plt.figure(figsize=(8, 4))
hour_counts.plot(kind="bar")
plt.xlabel("Hour (UTC)")
plt.ylabel("Number of observations")
plt.title("Distribution of observation hours (IBTrACS)")
plt.grid(axis="y")
plt.tight_layout()
plt.show()

In [ ]:
basins = sorted(df_ibtracs["basin"].unique())

for basin in basins:
    df_b = df_ibtracs[df_ibtracs["basin"] == basin]

    month_counts = (
        df_b["time_stamp"]
        .dt.month
        .value_counts()
        .sort_index()
    )

    plt.figure(figsize=(8, 4))
    month_counts.plot(kind="bar")
    plt.xlabel("Month")
    plt.ylabel("Number of observations")
    plt.title(f"Monthly distribution – Basin {basin}")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()


# Bouding Box

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime 

BASIN_BBOX = {
    "NA": [60, -100, 0, -10],
    "EP": [40, -160, 0, -80],
    "WP": [50, 100, 0, 180],
    "NI": [30, 40, -5, 100],
    "SI": [0, 20, -40, 100],
    "SP": [0, -160, -40, -120],
    "SA": [0, -60, -40, 20],
}

BASIN_COLORS = {
    "NA": "#1f77b4",  
    "EP": "#ff7f0e",  
    "WP": "#2ca02c",  
    "NI": "#d62728",  
    "SI": "#9467bd",  
    "SP": "#8c564b",  
    "SA": "#e377c2",  
}



def plot_basin_bounding_boxes_with_template(basin_bbox_dict):

    df_dummy = pd.DataFrame({
        "lon": [0],
        "lat": [0],
        "sid": ["_dummy_"],
        "name": ["dummy"]
    })

    fig = px.line_geo(
        df_dummy,
        lon="lon",
        lat="lat",
        projection="natural earth",
        title="Cyclone basin bounding boxes",
    )

    fig.data = []

    for basin, bbox in basin_bbox_dict.items():
        north, west, south, east = bbox
        color = BASIN_COLORS.get(basin, "black")

        lats = [south, north, north, south, south]
        lons = [west, west, east, east, west]

        fig.add_trace(
            go.Scattergeo(
                lon=lons,
                lat=lats,
                mode="lines",
                line=dict(width=2, color=color),
                name=basin,
                hoverinfo="text",
                text=(
                    f"Basin {basin}<br>"
                    f"N: {north}°<br>S: {south}°<br>"
                    f"W: {west}°<br>E: {east}°"
                ),
            )
        )

        fig.add_trace(
            go.Scattergeo(
                lon=[(west + east) / 2],
                lat=[(north + south) / 2],
                mode="text",
                text=[basin],
                textfont=dict(color=color, size=12),
                showlegend=False,
            )
        )

    fig.update_layout(
        legend_title_text="Basin",
        margin=dict(l=0, r=0, t=40, b=0),
        hovermode="closest",
    )

    fig.show()

plot_basin_bounding_boxes_with_template(BASIN_BBOX)

# ERA5

In [ ]:
# Paths
PATH = "../../data/processed/ibtracs_era5_20251217_1447.csv"

# Load data
df = pd.read_csv(PATH, parse_dates=["Timestamp"])

print("Dataset shape:", df.shape)
print("Years:", sorted(df["Timestamp"].dt.year.unique()))

df

In [ ]:
METEO_VARS = [
    "ERA5_Temp_2m_Kelvin",
    "ERA5_Pressure_MSL_hPa",
    "ERA5_Wind_U_Component",
    "ERA5_Wind_V_Component",
]

def summarize_year(df, year):
    year = int(year)
    df_y = df[df["Timestamp"].dt.year == year]

    if df_y.empty:
        print(f"\n===== SUMMARY {year} =====")
        print("No data")
        return

    print(f"\n===== SUMMARY {year} =====")
    print("Observations:", len(df_y))
    print("Cyclones:", df_y["Storm_ID"].nunique())
    print(
        "Period:",
        df_y["Timestamp"].min(),
        "→",
        df_y["Timestamp"].max(),
    )

    print("\nObservations per cyclone:")
    print(df_y.groupby("Storm_ID").size().describe())

    print("\nMeteorological variables:")
    display(df_y[METEO_VARS].describe())

for year in sorted(df["Timestamp"].dt.year.unique()):
    summarize_year(df, year)


In [ ]:
def check_nans_year(df, year):
    df_y = df[df["Timestamp"].dt.year == year]

    print(f"\nNaN check {year}")
    if df_y.empty:
        print("No data")
        return

    print(df_y[METEO_VARS].isna().mean())


for year in sorted(df["Timestamp"].dt.year.unique()):
    check_nans_year(df, year)


In [ ]:
def plot_cyclone_tracks_year(df, year):
    df_y = (
        df[df["Timestamp"].dt.year == year]
        .sort_values("Timestamp")
        .copy()
    )

    if df_y.empty:
        print(f"No data for {year}")
        return

    fig = px.line_geo(
        df_y,
        lon="Longitude",
        lat="Latitude",
        color="Storm_ID",
        hover_name="Storm_Name",
        projection="natural earth",
        title=f"Cyclone trajectories – {year}",
    )

    fig.update_traces(
        line=dict(width=2),
        hovertemplate=
        "<b>Cyclone:</b> %{customdata[0]}<br>"
        "<b>Date:</b> %{customdata[1]}<br>"
        "<b>Lat:</b> %{lat:.2f}°<br>"
        "<b>Lon:</b> %{lon:.2f}°<br><br>"
        "<b>IBTrACS</b><br>"
        "Wind: %{customdata[2]} kt<br>"
        "Pressure: %{customdata[3]} hPa<br><br>"
        "<b>ERA5</b><br>"
        "2m Temp: %{customdata[4]:.1f} K<br>"
        "MSLP: %{customdata[5]:.1f} hPa<br>"
        "U10: %{customdata[6]:.1f} m/s<br>"
        "V10: %{customdata[7]:.1f} m/s"
        "<extra></extra>",
        customdata=df_y[[
            "Storm_Name",
            "Timestamp",
            "Observed_Wind_Max_Knots",
            "Observed_Pressure_Min_mb",
            "ERA5_Temp_2m_Kelvin",
            "ERA5_Pressure_MSL_hPa",
            "ERA5_Wind_U_Component",
            "ERA5_Wind_V_Component",
        ]].values
    )

    fig.update_layout(
        legend_title_text="Storm ID",
        margin=dict(l=0, r=0, t=40, b=0),
        hovermode="closest",
    )

    fig.show()

plot_cyclone_tracks_year(df, 2022)
plot_cyclone_tracks_year(df, 2023)
plot_cyclone_tracks_year(df, 2024)

In [ ]:
def count_ibtracs_cyclones_year(df_ibtracs, year): 
    df_y = df_ibtracs[df_ibtracs["time_stamp"].dt.year == year] 
    return df_y["sid"].nunique()

def count_era5_cyclones_year(df_era5, year):
    df_y = df_era5[df_era5["Timestamp"].dt.year == year]
    return df_y["Storm_ID"].nunique()

def count_ibtracs_obs_year(df_ibtracs, year): 
    df_y = df_ibtracs[df_ibtracs["time_stamp"].dt.year == year] 
    return len(df_y)

def count_era5_obs_year(df_era5, year):
    df_y = df_era5[df_era5["Timestamp"].dt.year == year]
    return len(df_y)


rows = []

for year in years:
    n_cyc_ib = count_ibtracs_cyclones_year(df_ibtracs, year)
    n_cyc_era = count_era5_cyclones_year(df, year)

    n_obs_ib = count_ibtracs_obs_year(df_ibtracs, year)
    n_obs_era = count_era5_obs_year(df, year)

    rows.append({
        "Year": year,
        "Cyclones_IBTrACS": n_cyc_ib,
        "Cyclones_with_ERA5": n_cyc_era,
        "Cyclone_Coverage_Ratio": (
            n_cyc_era / n_cyc_ib if n_cyc_ib > 0 else np.nan
        ),
        "Observations_IBTrACS": n_obs_ib,
        "Observations_with_ERA5": n_obs_era,
        "Observation_Coverage_Ratio": (
            n_obs_era / n_obs_ib if n_obs_ib > 0 else np.nan
        ),
    })

df_coverage_full = pd.DataFrame(rows)
df_coverage_full



In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(8, 4))
plt.hist(df["ERA5_Position_Error_km"], bins=10, range=(0, 50))
plt.xlabel("ERA5 spatial error (km)")
plt.ylabel("Number of observations")
plt.title("ERA5 spatial error (50 km)")
plt.grid(True)
plt.tight_layout()
plt.show()


Un pas de grille ERA5 de 0.25° à 0.5° correspond à une résolution spatiale de l’ordre de 20 à 50 km selon la latitude, ce qui rend les erreurs spatiales observées (médiane ≈ 20 km, maximum ≈ 30 km) parfaitement cohérentes avec un échantillonnage nearest-neighbor.